In [ ]:
# df for question answer file csv contains 1 Lakh data points
import pandas as pd
df = pd.read_csv("/kaggle/input/qna-1l/cleaned_qna.csv")

In [107]:
# to read images.csv from vr-miniproject-2 or one can upload it manually and then copy the relative path
file_path = pd.read_csv("/kaggle/input/vr-miniproject-2/abo-images-small/images/metadata/images.csv")

In [108]:
# merging qna with images
final_df = pd.merge(df, file_path, on='image_id', how='left')

In [109]:
# to select specific col
final_df = final_df[["image_id","path","question","answer"]]

In [110]:
# adjusting the path of images according to kaggle
import os
image_folder = '/kaggle/input/vr-miniproject-2/abo-images-small/images/small/'
final_df['path'] = final_df['path'].apply(lambda p: os.path.join(image_folder, p))

In [111]:
# install bert score package
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [112]:
# install bart score
!git clone https://github.com/neulab/BARTScore.git
import sys
sys.path.append('/kaggle/working/BARTScore')

fatal: destination path 'BARTScore' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [113]:
# requiered classes to do fine tuning
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [114]:
# just to run some sample to test if the model is working or not, if one is sure one can work just remove .iloc[0:1000]
df = final_df.iloc[0:1000]

In [115]:
# change question and answert to lower case
df["question"] = df["question"].str.lower()
df["answer"] = df["answer"].str.lower()

/tmp/ipykernel_35/821518018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question"] = df["question"].str.lower()
/tmp/ipykernel_35/821518018.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["answer"] = df["answer"].str.lower()


In [138]:
# in this you can change the batch size so that it does not take time but to note that increasing batch size too much gives some issue like memory issue
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, ViltForQuestionAnswering, BitsAndBytesConfig, ViltProcessor
from peft import LoraConfig, get_peft_model, TaskType
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

# load model
quant_config = BitsAndBytesConfig(load_in_8bit=True)
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained(
    "dandelin/vilt-b32-finetuned-vqa",
    device_map="auto",  # Automatically allocates model across devices
    low_cpu_mem_usage=True
)
for name, module in model.named_modules():
    # print("hello")
    print(name)
# to create dataset for vqa for fine tuning
class VQADataset(Dataset):
    def __init__(self, dataframe, processor, max_target_length=4):
        self.dataframe = dataframe
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        question = self.dataframe.iloc[idx]["question"]
        image_path = self.dataframe.iloc[idx]["path"]
        answer = self.dataframe.iloc[idx]["answer"]

    
        image = Image.open(image_path).convert("RGB")
        prompt = f"Answer in one word only: {question}"
    
        # Step 1: Process image
        image_encoding = self.processor.image_processor(
            images=image,
            return_tensors="pt",
            do_resize=True,
            size={"height": 384, "width": 384}
        )
    
        # Step 2: Process text
        text_encoding = self.processor.tokenizer(
            prompt,
            padding="max_length",
            truncation=True,
            max_length=50,
            return_tensors="pt"
        )
    
        # Step 3: Combine and squeeze batch dim
        encoding = {
            "input_ids": text_encoding["input_ids"].squeeze(0),
            "attention_mask": text_encoding["attention_mask"].squeeze(0),
            "pixel_values": image_encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(self.label2id[answer])
        }
    
        return encoding


# Collate function
def collate_fn(batch):
    return {
        key: torch.stack([item[key] for item in batch])
        for key in batch[0]
    }

# Data split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_dataset = VQADataset(train_df, processor)
val_dataset = VQADataset(val_df, processor)

# one can change batch size inorder to run it faster (if it's very large then leads to out of memory issue)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)



vilt
vilt.embeddings
vilt.embeddings.text_embeddings
vilt.embeddings.text_embeddings.word_embeddings
vilt.embeddings.text_embeddings.position_embeddings
vilt.embeddings.text_embeddings.token_type_embeddings
vilt.embeddings.text_embeddings.LayerNorm
vilt.embeddings.text_embeddings.dropout
vilt.embeddings.patch_embeddings
vilt.embeddings.patch_embeddings.projection
vilt.embeddings.token_type_embeddings
vilt.embeddings.dropout
vilt.encoder
vilt.encoder.layer
vilt.encoder.layer.0
vilt.encoder.layer.0.attention
vilt.encoder.layer.0.attention.attention
vilt.encoder.layer.0.attention.attention.query
vilt.encoder.layer.0.attention.attention.key
vilt.encoder.layer.0.attention.attention.value
vilt.encoder.layer.0.attention.attention.dropout
vilt.encoder.layer.0.attention.output
vilt.encoder.layer.0.attention.output.dense
vilt.encoder.layer.0.attention.output.dropout
vilt.encoder.layer.0.intermediate
vilt.encoder.layer.0.intermediate.dense
vilt.encoder.layer.0.intermediate.intermediate_act_fn
vi

In [87]:
# this is used to fund module names which can be used for fine tuning as it should be all linear
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

vilt.encoder.layer.0.attention.attention.query
vilt.encoder.layer.0.attention.attention.key
vilt.encoder.layer.0.attention.attention.value
vilt.encoder.layer.0.attention.output.dense
vilt.encoder.layer.0.intermediate.dense
vilt.encoder.layer.0.output.dense
vilt.encoder.layer.1.attention.attention.query
vilt.encoder.layer.1.attention.attention.key
vilt.encoder.layer.1.attention.attention.value
vilt.encoder.layer.1.attention.output.dense
vilt.encoder.layer.1.intermediate.dense
vilt.encoder.layer.1.output.dense
vilt.encoder.layer.2.attention.attention.query
vilt.encoder.layer.2.attention.attention.key
vilt.encoder.layer.2.attention.attention.value
vilt.encoder.layer.2.attention.output.dense
vilt.encoder.layer.2.intermediate.dense
vilt.encoder.layer.2.output.dense
vilt.encoder.layer.3.attention.attention.query
vilt.encoder.layer.3.attention.attention.key
vilt.encoder.layer.3.attention.attention.value
vilt.encoder.layer.3.attention.output.dense
vilt.encoder.layer.3.intermediate.dense
vilt.e

In [139]:
# PEFT: LoRA configuration 
# one can change here r, alpha, dropout, bias, target_modules (for target_modules select from above layers if u want additional modules)

# lora_config = LoraConfig(
#     r=128,
#     lora_alpha=128,
#     lora_dropout=0.05,
#     bias="none",
#     target_modules=["attention.self.query", "attention.self.value"]
# )
peft_config = LoraConfig(
    
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 442,368 || all params: 118,030,905 || trainable%: 0.3748


In [137]:
print(df.columns)  # or whatever your DataFrame is named

Index(['image_id', 'path', 'question', 'answer'], dtype='object')


In [140]:
# was trained some more time like 30 iteratins
optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)  # Slightly increased learning rate


device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model = model.to(torch.float32)
model.train()

# can change number of iteration  here
for epoch in range(20): 
    print(f"\nEpoch {epoch + 1}")
    total_loss = 0

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        pixel_values = batch["pixel_values"].to(device)  # defaults to float32
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Mask padding tokens in labels to avoid loss explosion
        labels[labels == processor.tokenizer.pad_token_id] = -100

        # Check for NaN/Inf in pixel values and labels before forward pass
        if torch.isnan(pixel_values).any() or torch.isinf(pixel_values).any():
            print("[Error] NaN/Inf in pixel values")
            continue
        if torch.isnan(labels).any() or torch.isinf(labels).any():
            print("[Error] NaN/Inf in labels")
            continue

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            labels=labels
        )
        if torch.isnan(outputs.loss).any() or torch.isinf(outputs.loss).any():
            print(f"[Error] NaN/Inf loss at step {step}")
            print(f"Logits: {outputs.logits}")
            continue
        # Check for NaN/Inf loss
        
        # Decode predictions occasionally
        if step == 0:
            print(f"[Pixel Stats] min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

            predicted_ids = outputs.logits.argmax(dim=-1)
            predicted_texts = processor.batch_decode(predicted_ids, skip_special_tokens=True)
            decoded_labels = labels.clone()
            decoded_labels[decoded_labels == -100] = processor.tokenizer.pad_token_id
            ground_truth_texts = processor.batch_decode(decoded_labels, skip_special_tokens=True)

            input_prompts = processor.batch_decode(input_ids, skip_special_tokens=True)
            for i in range(min(2, len(predicted_texts))):
                print(f"\n[Prompt]      {input_prompts[i]}")
                print(f"[Prediction]  {predicted_texts[i]}")
                print(f"[GroundTruth] {ground_truth_texts[i]}")

        # Compute loss
        loss = outputs.loss

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"[Warning] NaN/Inf loss at step {step} — skipping batch")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)  # Lower clipping norm to prevent gradients explosion
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        if step % 10 == 0:
            print(f"  Step {step}: Loss = {loss.item():.4f}")

    # Print average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} average loss: {avg_loss:.4f}")
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_step, val_batch in enumerate(val_loader):
            input_ids = val_batch["input_ids"].to(device)
            pixel_values = val_batch["pixel_values"].to(device)
            attention_mask = val_batch["attention_mask"].to(device)
            labels = val_batch["labels"].to(device)

            labels[labels == processor.tokenizer.pad_token_id] = -100

            if torch.isnan(pixel_values).any() or torch.isinf(pixel_values).any():
                print("[Validation Error] NaN/Inf in pixel values")
                continue
            if torch.isnan(labels).any() or torch.isinf(labels).any():
                print("[Validation Error] NaN/Inf in labels")
                continue

            outputs = model(
                input_ids=input_ids,
                pixel_values=pixel_values,
                attention_mask=attention_mask,
                labels=labels
            )

            if torch.isnan(outputs.loss).any() or torch.isinf(outputs.loss).any():
                print(f"[Validation Error] NaN/Inf loss at val_step {val_step}")
                continue

            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation loss after epoch {epoch + 1}: {avg_val_loss:.4f}")
    model.train()


Epoch 1


ValueError: The `size` dictionary must contain the key `shortest_edge`. Got dict_keys(['height', 'width'])

In [21]:
# save the model to output directory
from transformers import AutoProcessor

# Save model
model.save_pretrained("blip2_trained_lora_model")

# Save processor (tokenizer + image processor)
processor.save_pretrained("blip2_trained_model")

['blip2_trained_model/processor_config.json']

In [22]:
# please download the blip2_trained_model or else all training data will be lost
import shutil

# Zip the folder
shutil.make_archive("blip2_trained_lora_model", 'zip', "blip2_trained_lora_model")

'/kaggle/working/blip2_trained_lora_model.zip'

In [ ]:
# main blip code to generate words for prediction with taking last word
# for index, row in final_df.head(5).iterrows(): 
# in the above line change the number of samples of like 5 to 10 and wish to run for whole then just put: for index, row in final_df.iterrows(): 
# and if one does not want to see prediction and all comment down the print commands in for loop
import os
import torch
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration, logging
from bert_score import score as bertscore
from bart_score import BARTScorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import Blip2ForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from peft import PeftModel
import torch

# the warning which u will see below is because we are not training all the target modules so lora model does not have all the values so it gaves warning that these modeules are missing
import warnings
warnings.filterwarnings("ignore", message="Found missing adapter keys while loading the checkpoint")

# Suppress logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
logging.set_verbosity_error()

# if using downloaded lora weights change the finetuned_path to the path of uploaded model
finetuned_path = "/kaggle/working/blip2_trained_lora_model"
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    device_map="auto",
    quantization_config=quant_config,
)

model = PeftModel.from_pretrained(base_model, finetuned_path)

# Load processor
processor = AutoProcessor.from_pretrained("Salesforce/blip2-flan-t5-xl")

model.eval()


bart_scorer = BARTScorer(device="cuda" if torch.cuda.is_available() else "cpu", checkpoint='facebook/bart-large-cnn')

# Stats
true_labels = []
pred_labels = []
bert_P_all = []
bert_R_all = []
bert_F1_all = []
bart_scores_all = []

# Loop through samples (remove `.head(n)` to run full)
for index, row in final_df.iloc[20000:20040].iterrows():
    try:
        question = row["question"]
        true_ans = row["answer"].strip().lower()
        image_path = row["path"]

        image = Image.open(image_path).convert("RGB")

        prompt = f"Answer in one word only: {question}"
        inputs = processor(image, prompt, return_tensors="pt").to(model.device, torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=4)
        pred_ans = processor.decode(generated_ids[0], skip_special_tokens=True).strip().lower()
        # pred_ans = pred_ans.split()[-1]
        

        true_labels.append(true_ans)
        pred_labels.append(pred_ans)

        # BERTScore
        P, R, F1 = bertscore([pred_ans], [true_ans], lang="en", rescale_with_baseline=True)
        bart = bart_scorer.score([pred_ans], [true_ans])[0]

        bert_P_all.append(P[0].item())
        bert_R_all.append(R[0].item())
        bert_F1_all.append(F1[0].item())
        bart_scores_all.append(bart)

        print(f"\nRow {index} | Image: {row['image_id']}")
        print(f"Q: {question}\nPredicted: {pred_ans} | Actual: {true_ans}")
        print(f"Exact Match: {pred_ans == true_ans}")
        print(f"BERTScore - P: {P[0].item():.4f} | R: {R[0].item():.4f} | F1: {F1[0].item():.4f} | BARTScore: {bart:.4f}")

    except Exception as e:
        print(f"Skipping row {index} due to error: {e}")
        continue

# Classification Metrics (Exact Match)
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
recall = recall_score(true_labels, pred_labels, average='macro', zero_division=0)
f1 = f1_score(true_labels, pred_labels, average='macro', zero_division=0)

# Semantic Metrics
avg_bert_P = sum(bert_P_all) / len(bert_P_all) if bert_P_all else 0
avg_bert_R = sum(bert_R_all) / len(bert_R_all) if bert_R_all else 0
avg_bert_F1 = sum(bert_F1_all) / len(bert_F1_all) if bert_F1_all else 0
avg_bart = sum(bart_scores_all) / len(bart_scores_all) if bart_scores_all else 0

# Final summary
print("\n===== FINAL SUMMARY =====")
print(f"Classification Accuracy:       {accuracy * 100:.2f}%")
print(f"Precision (macro avg):         {precision * 100:.2f}%")
print(f"Recall (macro avg):            {recall * 100:.2f}%")
print(f"F1 Score (macro avg):          {f1 * 100:.2f}%")

print(f"\nAvg BERTScore - P:             {avg_bert_P * 100:.2f}%")
print(f"Avg BERTScore - R:             {avg_bert_R * 100:.2f}%")
print(f"Avg BERTScore - F1:            {avg_bert_F1 * 100:.2f}%")
print(f"Avg BARTScore:                 {avg_bart:.4f}")


In [24]:
# this was done to check for df not used for training
true_labels = []
pred_labels = []
bert_P_all = []
bert_R_all = []
bert_F1_all = []
bart_scores_all = []

# Loop through samples (remove `.head(n)` to run full)
for index, row in final_df.iloc[10001:10041].iterrows():
    try:
        question = row["question"]
        true_ans = row["answer"].strip().lower()
        image_path = row["path"]

        image = Image.open(image_path).convert("RGB")

        prompt = f"Answer in one word only: {question}"
        inputs = processor(image, prompt, return_tensors="pt").to(model.device, torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=4)
        pred_ans = processor.decode(generated_ids[0], skip_special_tokens=True).strip().lower()
        # pred_ans = pred_ans.split()[-1]
        

        true_labels.append(true_ans)
        pred_labels.append(pred_ans)

        # BERTScore
        P, R, F1 = bertscore([pred_ans], [true_ans], lang="en", rescale_with_baseline=True)
        bart = bart_scorer.score([pred_ans], [true_ans])[0]

        bert_P_all.append(P[0].item())
        bert_R_all.append(R[0].item())
        bert_F1_all.append(F1[0].item())
        bart_scores_all.append(bart)

        print(f"\nRow {index} | Image: {row['image_id']}")
        print(f"Q: {question}\nPredicted: {pred_ans} | Actual: {true_ans}")
        print(f"Exact Match: {pred_ans == true_ans}")
        print(f"BERTScore - P: {P[0].item():.4f} | R: {R[0].item():.4f} | F1: {F1[0].item():.4f} | BARTScore: {bart:.4f}")

    except Exception as e:
        print(f"Skipping row {index} due to error: {e}")
        continue

# Classification Metrics (Exact Match)
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
recall = recall_score(true_labels, pred_labels, average='macro', zero_division=0)
f1 = f1_score(true_labels, pred_labels, average='macro', zero_division=0)

# Semantic Metrics
avg_bert_P = sum(bert_P_all) / len(bert_P_all) if bert_P_all else 0
avg_bert_R = sum(bert_R_all) / len(bert_R_all) if bert_R_all else 0
avg_bert_F1 = sum(bert_F1_all) / len(bert_F1_all) if bert_F1_all else 0
avg_bart = sum(bart_scores_all) / len(bart_scores_all) if bart_scores_all else 0

# Final summary
print("\n===== FINAL SUMMARY =====")
print(f"Classification Accuracy:       {accuracy * 100:.2f}%")
print(f"Precision (macro avg):         {precision * 100:.2f}%")
print(f"Recall (macro avg):            {recall * 100:.2f}%")
print(f"F1 Score (macro avg):          {f1 * 100:.2f}%")

print(f"\nAvg BERTScore - P:             {avg_bert_P * 100:.2f}%")
print(f"Avg BERTScore - R:             {avg_bert_R * 100:.2f}%")
print(f"Avg BERTScore - F1:            {avg_bert_F1 * 100:.2f}%")
print(f"Avg BARTScore:                 {avg_bart:.4f}")



Row 10001 | Image: 316E26-HJUL
Q: What is the product?
Predicted: bag | Actual: pulses
Exact Match: False
BERTScore - P: 0.9948 | R: 0.9948 | F1: 0.9948 | BARTScore: -5.9466

Row 10002 | Image: 316GBAhhv9L
Q: What is the bottle's top?
Predicted: lid | Actual: white
Exact Match: False
BERTScore - P: 0.9884 | R: 0.9884 | F1: 0.9884 | BARTScore: -7.3317

Row 10003 | Image: 316GBAhhv9L
Q: What is the liquid's hue?
Predicted: green | Actual: yellow
Exact Match: False
BERTScore - P: 0.8104 | R: 0.8104 | F1: 0.8107 | BARTScore: -6.0827

Row 10004 | Image: 316GBAhhv9L
Q: What is the container's form?
Predicted: cylindrical | Actual: bottle
Exact Match: False
BERTScore - P: -0.6180 | R: 0.4482 | F1: -0.1416 | BARTScore: -6.3076

Row 10005 | Image: 316IM3U2JQL
Q: What garment is displayed?
Predicted: jacket | Actual: jacket
Exact Match: True
BERTScore - P: 1.0000 | R: 1.0000 | F1: 1.0000 | BARTScore: -1.8321

Row 10006 | Image: 316IM3U2JQL
Q: What color is the coat?
Predicted: black | Actual: b